<a href="https://colab.research.google.com/github/durgaprasaddp72/spell_checker/blob/main/Spell_Cheacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from tqdm import tqdm#for calculating the timecomplexity of the for loop

## Finding the Unique Words

In [2]:
with open("big.txt",'r')as f:
  lines=f.readlines()
  words=[]
  for line in lines:
    words+=re.findall('\w+',line.lower())
print(len(words))
vocab=list(set(words))
print(len(vocab))

1115585
32198


## Finding the Probability Distribution

In [3]:
word_probability={}
for word in tqdm(vocab):
  word_probability[word]=float(words.count(word)/len(words))


100%|██████████| 32198/32198 [12:57<00:00, 41.40it/s]


## Text Preprocessing

#### splitting

In [4]:
def split(word):
  parts=[]
  for i in range(len(word)+1):
    parts+=[(word[:i],word[i:])]
  return parts

In [5]:
split("durga")

[('', 'durga'),
 ('d', 'urga'),
 ('du', 'rga'),
 ('dur', 'ga'),
 ('durg', 'a'),
 ('durga', '')]

### delete

In [6]:
def delete(word):
  output=[]
  for l,r in split(word):
    output.append(l+r[1:])
  return output
delete("loave")

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

### swap

In [7]:
def swap(word):
  output=[]
  for l,r in split(word):
    if (len(r)>1):
      output.append(l+r[1]+r[0]+r[2:])
  return output
swap("lvoe")

['vloe', 'love', 'lveo']

### replace

In [8]:
def replace(word):
  characters="".join([chr(i) for i in range(97,123)])#a-z assci(97-122)
  output=[]
  for l,r in split(word):
    for char in characters:
      output.append(l+char+r[1:])
  return output

len(replace('lave'))


130

### insert

In [9]:
def insert(word):
  characters="".join([chr(i) for i in range(97,123)])
  output=[]
  for l,r in split(word):
    for char in characters:
      output.append(l+char+r)
  return output

len(insert('prsd'))
#len(insert('lve'))


130

## combining all possible words

In [12]:
def edit(word):
  return list(set(insert(word)+delete(word)+swap(word)+replace(word)))

## prdicting the word

In [11]:
import pandas as pd

In [13]:
def spell_check_edit_1(word,count=5):
  output=[]
  suggested_words=edit(word)
  for word in suggested_words:
    if word in word_probability.keys():
      output.append([word,word_probability[word]])
  return list(pd.DataFrame(output,columns=["word","prob"]).sort_values(by="prob",ascending=False).head(count)["word"].values)

In [14]:
spell_check_edit_1("famili")

['family']

some times the one edit not work well then we are going to use multiple edits inorder to find possible words.

In [18]:
def spell_cheak_edit_2(word,count=5):
  output=[]
  suggested_words=edit(word)#level1 edit
  for e1 in edit(word):
    suggested_words+=edit(e1)#level 2 edit
  suggested_words=list(set(suggested_words))

  for word in suggested_words:
    if word in word_probability.keys():
      output.append([word,word_probability[word]])
  return list(pd.DataFrame(output,columns=["word","probability"]).sort_values(by="word",ascending=False).head(count)["word"].values)
spell_cheak_edit_2("fameli")

['namely', 'lamely', 'family', 'famed', 'fame']